In [7]:
import pandas as pd
import os

In [8]:
# Specify the data folder
data_folder = "./../processed_data"

# Read each CSV file into a separate DataFrame
df = pd.read_csv(os.path.join(data_folder, 'games.csv'))

# Baseline

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(df["weekly_attendance"], df["avg_season_attendance_prev"])
rmse = mean_squared_error(df["weekly_attendance"], df["avg_season_attendance_prev"], squared=False)
mdae = np.median(np.abs(df["weekly_attendance"] - df["avg_season_attendance_prev"]))

print(f"MAE: {mae:.2f}")
print(f"MDAE: {mdae:.2f}")
print(f"RMSE: {rmse:.2f}")

MAE: 4440.49
MDAE: 3147.47
RMSE: 6461.32


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Model

In [10]:
df.columns

Index(['year', 'week', 'home_team_name', 'away_team_name', 'day', 'date',
       'time', 'weekly_attendance', 'weekly_attendance_ratio',
       'prev_game_attendance', 'avg_season_attendance_prev',
       'home_team_superbowl_winner_last_season',
       'away_team_superbowl_winner_last_season',
       'home_team_playoffs_last_season', 'away_team_playoffs_last_season',
       'home_team_wins_last_3', 'away_team_wins_last_3', 'pts_home',
       'pts_away', 'yds_home', 'yds_away', 'turnovers_home', 'turnovers_away',
       'zscore_pts_home', 'zscore_yds_home', 'zscore_turnovers_home',
       'zscore_pts_away', 'zscore_yds_away', 'zscore_turnovers_away'],
      dtype='object')

In [15]:
import torch
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoNormal
from pyro.optim import ClippedAdam

class LatentAttendanceModel(PyroModule):
    def __init__(self, x_dim, T):
        super().__init__()
        self.x_dim = x_dim
        self.T = T

    def model(self, y, x, avg_attendance, z_lag):
        tau = pyro.sample("tau", dist.HalfNormal(10.0))
        sigma = pyro.sample("sigma", dist.HalfNormal(10.0))

        beta_0 = pyro.sample("beta_0", dist.Normal(0., 10.))
        beta_1 = pyro.sample("beta_1", dist.Normal(0., 1.))  # autoregressive effect
        beta_avg = pyro.sample("beta_avg", dist.Normal(1., 0.1))
        beta_x = pyro.sample("beta_x", dist.Normal(0., 1.).expand([self.x_dim]).to_event(1))

        z = torch.zeros(self.T)

        for t in range(self.T):
            x_t = x[t]
            if t == 0:
                z_prev = z_lag[t]  # could also just set to 0
            else:
                z_prev = z[t - 1]

            mu_z = beta_0 + beta_1 * z_prev + beta_avg * avg_attendance[t] + torch.dot(x_t, beta_x)
            z[t] = pyro.sample(f"z_{t}", dist.Normal(mu_z, tau))

            pyro.sample(f"y_{t}", dist.Normal(z[t], sigma), obs=y[t])


In [16]:
# Convert your data to torch tensors
y_tensor = torch.tensor(df["weekly_attendance"].values, dtype=torch.float32)
x_tensor = torch.tensor(df[[
    "away_team_superbowl_winner_last_season",
    "home_team_playoffs_last_season",
    "away_team_playoffs_last_season",
    "home_team_wins_last_3",
    "away_team_wins_last_3",
    'zscore_pts_home', 'zscore_yds_home', 'zscore_turnovers_home',
    'zscore_pts_away', 'zscore_yds_away', 'zscore_turnovers_away'
]].values, dtype=torch.float32)

avg_att_tensor = torch.tensor(df["avg_season_attendance_prev"].values, dtype=torch.float32)
z_lag_tensor = torch.tensor(df["prev_game_attendance"].values, dtype=torch.float32)

T = len(y_tensor)
x_dim = x_tensor.shape[1]

# Define model
model = LatentAttendanceModel(x_dim=x_dim, T=T)

# AutoGuide for variational inference
guide = AutoNormal(model.model)

# Optimizer and SVI
optimizer = ClippedAdam({"lr": 0.01})
svi = SVI(model.model, guide, optimizer, loss=Trace_ELBO())

# Training loop
num_steps = 2000
for step in range(num_steps):
    loss = svi.step(y_tensor, x_tensor, avg_att_tensor, z_lag_tensor)
    if step % 200 == 0:
        print(f"Step {step} - ELBO: {loss:.2f}")


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.